In [ ]:
!python -m pip install pymed

In [ ]:
import re
import string
import pickle
import pandas as pd
from pymed import PubMed
import itertools

In [ ]:
#read data in AED_NEG line by line 
with open('ADE-NEG.txt') as f:
    content_NEG = f.readlines()
content_NEG = [x.strip() for x in content_NEG] 

list_NEG_query = [re.findall('\d+|$', content_NEG[i])[0] for i in range(len(content_NEG))]

#read data in AED_POS line by line 
with open('DRUG-AE.rel') as f:
    content_POS = f.readlines()
content_POS = [x.strip() for x in content_POS] 

list_POS_query = [re.findall('\d+|$', content_POS[i])[0] for i in range(len(content_POS))]

#set of unique article IDs
list_query = list(set(list_NEG_query + list_POS_query))
n_requests = len(list_query)
n_requests

2972

In [ ]:
def peek(iterable):
    try:
        first = next(iterable)
    except StopIteration:
        return None
    return first, itertools.chain([first], iterable)

In [ ]:
#from pymed import PubMed # Create a PubMed object that GraphQL can use to query
# Note that the parameters are not required but kindly requested by PubMed Central
# https://www.ncbi.nlm.nih.gov/pmc/tools/developers/

def take_article(query):
    pubmed = PubMed(tool="MyTool", email="my@email.address")
    results = pubmed.query(query, max_results=500)

    res = peek(results)

    if res is None:
        print('{} not found in Pudmed'.format(query))
        id = []
        title = []
        publication_date = []
        abstract = []
    else:
        #print('Not empty')
        results = pubmed.query(query, max_results=500)
        #print(results)
        for article in results:
            # Extract and format information from the article
            #article_id = article.pubmed_id
            #id = re.findall('\d+|$', f'{article_id}')[0]
            id = re.findall('\d+|$', article.pubmed_id)[0]
            title = article.title
            publication_date = article.publication_date
            abstract = article.abstract    # Show information about the article

            if article.keywords:
                if None in article.keywords:
                    article.keywords.remove(None)
                    keywords = '", "'.join(article.keywords)

    #id = re.findall('\d+|$', f'{article_id}')[0] #take only the article ID (first element)
    return id, publication_date, title, abstract

In [ ]:
columns=['articleID', 'publication_date', 'title', 'abstract']
df_list = pd.DataFrame(columns=columns)

for i in range(n_requests): #n_requests
  #query = re.findall('\d+|$', content[i])[0]   #[int(s) for s in content[i].split() if s.isdigit()]
  #label = re.findall('NEG', content[i])[0]
  #title.append(content[i][(content[i].find('NEG')+4):].replace("'", ""))

   df_list = df_list.append(pd.Series(take_article(list_query[i]), index=columns), ignore_index=True)

#SAVE data to csv
df_list.to_csv('data_articleID_abstract.csv')
#SAVE data to pickle
with open('data_articleID_abstract.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)